In [7]:
import pickle
import numpy as np
import pandas as pd
from time import time
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.metrics import roc_curve, auc
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from hpsklearn import HyperoptEstimator , tfidf, pca, any_text_preprocessing
from hyperopt import hp, tpe, fmin
from hyperopt import STATUS_OK, Trials
from hyperopt import space_eval
from sklearn.metrics import accuracy_score

ModuleNotFoundError: No module named 'hpsklearn'

In [25]:
with open('NHlegoverall.dict','rb') as f:
    leg_data = pickle.load(f)
with open('NHLegTimeSeriesLabels.list','rb') as f:
    leg_target = pickle.load(f)

In [26]:
doc_embeddings = pd.read_pickle("NHlegtext_embed_wBill.DF")
#print(doc_embeddings)
doc_embeddings.describe()

new_leg_data = {'event':leg_data['event'], 'bill_cat':leg_data['bill_cat'], 'chamber':leg_data['chamber'],
                'sponsor_vec':leg_data['sponsor_vec2'], 'time_t':leg_data['time_t'], 'year':leg_data['year'],
                'session':leg_data['session']}

#normalize the time data
for i, value in enumerate(new_leg_data['time_t']):
    new_leg_data['time_t'][i] = value/10 

#doesnt make a difference whether we make sessions and year categorical data instead of integers or not, so don't do it
# for i, value in enumerate(new_leg_data['session']):
#     new_leg_data['session'][i] = new_leg_data['session'][i]
#     new_leg_data['year'][i] = new_leg_data['year'][i]

#transform sponsor_vec to between 0 and 1
#for i, value in enumerate(new_leg_data['sponsor_vec']):
#    new_leg_data['sponsor_vec'][i] = 0.5*new_leg_data['sponsor_vec'][i] + 0.5

    
#set the default embedding to the average embedding from df.describe()
#default_embedding = {'doc_id':0, 'bill_id':0, 'dv1':-0.429835,'dv2':-0.348652,'dv3':-0.096175,'dv4':0.722217,'dv5':-0.080119,'dv6':0.331679,'dv7':0.886933,'dv8':0.934954,'dv9':-0.798512,'dv10':0.054533,'dv11':-0.028231,'dv12':0.561587,'dv13':-0.485424,'dv14':0.015666,'dv15':0.864729,'dv16':-0.697787}


leg_data = new_leg_data

In [27]:
leg_data_df = pd.DataFrame(leg_data)
leg_data_df = pd.get_dummies(leg_data_df)
leg_label_df = pd.DataFrame(leg_target)
leg_label_df = pd.get_dummies(leg_label_df)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(leg_data_df, leg_target, test_size=0.3,random_state=109, shuffle=False) # 70% training and 30% test

In [39]:
#define a performance metrics function (accuracy and percision and recall)
def perf_metric (y_test, y_pred):
#Accuracy
    x=0
    y=0
    acc=0
    for i in range(10205):
        if y_test[i] == y_pred[i]:
            x+=1
        y+=1
    acc=x/y*100
    print(x,y,len(y_pred),len(y_test))
    print('Accuracy:', acc)

#Precision and Recall
# passing keywords list
    nh_passing_keywords =['ought to pass' ,'ougut to pass with amendment','inexpedient to legislate','signed', 'law without signature', 'motion adopted', 'introducted and adopted']
    tp=0
    fp=0
    fn=0
    tn=0
    perc=0 
    recall= 0 
    pred_pass= [0]*len(y_pred)
    test_pass=[]
    
    for i in range(len(y_pred)):
        if y_pred[i] in nh_passing_keywords:
            pred_pass[i]==1
        else:
            pred_pass[i]==0
        
    for i in range(10205):
        if y_pred[i]==y_test[i] and pred_pass[i]==1:
            tp+=1
        elif y_pred[i]==y_test[i]:
            fp+=1
        elif y_pred[i]!=y_test[i]:
            fn+=1
        elif y_pred[i]==y_test[i] and pred_pass[i]==0:
            tn+=1
        
    if tp==0 and fp==0: 
        perc=0
    else:
        perc= tp/(tp+fp)  
          
    if tp==0 and fn==0: 
        recall=0
    else: 
        recall = tp/(tp+fn)

    print(tp,fp,fn,tn)
    print('Precision:', perc)
    print('Recall', recall)

In [10]:
#####################################  DecisionTreeClassifier #################################

In [31]:
#Create a Gaussian Classifier
clf = DecisionTreeClassifier()

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)
y_proba_log = clf.predict_log_proba(X_test)

/usr/local/lib/python3.5/site-packages/sklearn/tree/tree.py:864: RuntimeWarning: divide by zero encountered in log
  return np.log(proba)


In [40]:
perf_metric(y_test,y_pred)

4568 10205 10205 10205
Accuracy: 44.762371386575204
0 4568 5637 0
Precision: 0.0
Recall 0.0


In [61]:
clf = DecisionTreeClassifier()

#  preprocessing= any_text_preprocessing
estim = HyperoptEstimator(classifier=clf, algo=tpe.suggest, max_evals=5)
estim.fit(X_train.values, y_train)#X_train.values, y_train
#clf.fit(X_train,y_train)
y_pred = estim.predict(X_test)

100%|██████████| 1/1 [00:01<00:00,  1.77s/it, best loss: 0.6091978160436791]


In [62]:
perf_metric(y_test,y_pred)

4568 10205 10205 10205
Accuracy: 44.762371386575204
0 4568 5637 0
Precision: 0.0
Recall 0.0


In [54]:
clf = MultinomialNB()

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)
y_proba_log = clf.predict_log_proba(X_test)

In [55]:
perf_metric(y_test,y_pred)

5046 10205 10205 10205
Accuracy: 49.44634982851544
0 5046 5159 0
Precision: 0.0
Recall 0.0


In [57]:
clf = MultinomialNB()

estim = HyperoptEstimator(classifier=clf, algo=tpe.suggest, max_evals=2)
estim.fit(X_train.values, y_train)#X_train.values, y_train
y_pred = estim.predict(X_test)

100%|██████████| 1/1 [00:00<00:00,  1.82it/s, best loss: 0.50629987400252]


In [58]:
perf_metric(y_test,y_pred)

5046 10205 10205 10205
Accuracy: 49.44634982851544
0 5046 5159 0
Precision: 0.0
Recall 0.0


In [ ]:
clf = GaussianNB()

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)
y_proba_log = clf.predict_log_proba(X_test)

In [ ]:
clf=GaussianNB()

estim = HyperoptEstimator(classifier=clf, algo=mix_algo, max_evals=15)
estim.fit(X_train.values, y_train)#X_train.values, y_train
#clf.fit(X_train,y_train)
y_pred = estim.predict(X_test)
#y_proba = clf.predict_proba(X_test)

In [59]:
clf = RandomForestClassifier()

#y_pred=grid_search.pred(X_test)
#y_pred=grid_search.pred(X_test)


clf = RandomForestClassifier()
clf.fit(X_train, y_train)  
y_pred= clf.predict(X_test)

perf_metric(y_test, y_pred)

4821 10205 10205 10205
Accuracy: 47.24154826065654
0 4821 5384 0
Precision: 0.0
Recall 0.0


In [60]:
clf = RandomForestClassifier(
    min_samples_split=5, 
    criterion = 'gini', 
    max_features= 0.2, 
    max_depth= 10)

#y_pred=grid_search.pred(X_test)
#y_pred=grid_search.pred(X_test)


clf = RandomForestClassifier()
clf.fit(X_train, y_train)  
y_pred= clf.predict(X_test)

perf_metric(y_test, y_pred)

4799 10205 10205 10205
Accuracy: 47.02596766291034
0 4799 5406 0
Precision: 0.0
Recall 0.0


In [69]:
clf = MLPClassifier()
clf.fit(X_train, y_train)                         

y_pred=clf.predict(X_test)
y_proba = clf.predict_proba(X_test)

In [70]:
perf_metric(y_test, y_pred)

26 10205 10205 10205
Accuracy: 0.25477707006369427
0 26 10179 0
Precision: 0.0
Recall 0.0


In [65]:
clf = MLPClassifier()
estim = HyperoptEstimator(classifier=clf, algo=tpe.suggest, max_evals=2)

estim.fit(X_train.values, y_train)

y_pred = estim.predict(X_test)

  0%|          | 0/1 [00:00<?, ?it/s, best loss: ?]

/usr/local/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)



100%|██████████| 1/1 [00:25<00:00, 25.55s/it, best loss: 0.5083998320033599]


In [66]:
perf_metric(y_test, y_pred)

475 10205 10205 10205
Accuracy: 4.654581087702106
0 475 9730 0
Precision: 0.0
Recall 0.0
